In [5]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
from bq_helper import BigQueryHelper
%load_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="My Project-bbdce7b1712b.json"

bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")

query="""
SELECT * FROM `bigquery-public-data.san_francisco.sfpd_incidents` 
"""

df = bq_assistant.query_to_pandas(query)


The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [11]:
df[df['unique_key']==80460270]

,unique_key,category,descript,dayofweek,pddistrict,resolution,address,longitude,latitude,location,pdid,timestamp
9,80460270,DISORDERLY CONDUCT,COMMITTING PUBLIC NUISANCE,Friday,PARK,"ARREST, BOOKED",HAIGHT ST / BUENAVISTAWEST AV,-122.443312,37.77047,"(37.7704697548539, -122.443311949602)",8046027019010,2008-05-02 14:31:00+00:00
280,80460270,DRUNKENNESS,UNDER INFLUENCE OF ALCOHOL IN A PUBLIC PLACE,Friday,PARK,"ARREST, BOOKED",HAIGHT ST / BUENAVISTAWEST AV,-122.443312,37.77047,"(37.7704697548539, -122.443311949602)",8046027019090,2008-05-02 14:31:00+00:00
942761,80460270,DRUG/NARCOTIC,POSSESSION OF MARIJUANA,Friday,PARK,"ARREST, BOOKED",HAIGHT ST / BUENAVISTAWEST AV,-122.443312,37.77047,"(37.7704697548539, -122.443311949602)",8046027016010,2008-05-02 14:31:00+00:00


In [2]:
df.nunique()

unique_key    1713368
category           39
descript          915
dayofweek           7
pddistrict         11
resolution         17
address         25123
longitude       60392
latitude        59107
location        60768
pdid          2170785
timestamp      960696
dtype: int64

In [10]:
df.nunique()

unique_key     9442
category         36
descript        502
dayofweek         7
pddistrict        1
resolution       16
address        1145
longitude      1832
latitude       1819
location       1838
pdid          10000
timestamp      9336
dtype: int64

In [12]:
set(df['resolution'])

{'ARREST, BOOKED',
 'ARREST, CITED',
 'CLEARED-CONTACT JUVENILE FOR MORE INFO',
 'COMPLAINANT REFUSES TO PROSECUTE',
 'DISTRICT ATTORNEY REFUSES TO PROSECUTE',
 'EXCEPTIONAL CLEARANCE',
 'JUVENILE ADMONISHED',
 'JUVENILE BOOKED',
 'JUVENILE CITED',
 'JUVENILE DIVERTED',
 'LOCATED',
 'NONE',
 'NOT PROSECUTED',
 'PROSECUTED BY OUTSIDE AGENCY',
 'PSYCHOPATHIC CASE',
 'UNFOUNDED'}

In [13]:
pip install --upgrade google-cloud-bigquery[pandas]



The following command must be run outside of the IPython shell:

    $ pip install --upgrade google-cloud-bigquery[pandas]

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [14]:
%pip install --upgrade google-cloud-bigquery[pandas]


The following command must be run outside of the IPython shell:

    $ pip install --upgrade google-cloud-bigquery[pandas]

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/
